In [1]:
# Streamlit dashboard para GoFundMe - Nicolás Ton
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

# Simulación de datos
np.random.seed(42)
categories = ['Salud', 'Educación', 'Desastres Naturales', 'Animales', 'Comunidad']
countries = ['Argentina', 'USA', 'Brasil', 'México', 'España']

n_campaigns = 300
campaigns = pd.DataFrame({
    'id': range(1, n_campaigns + 1),
    'nombre': [f'Campaña {i}' for i in range(1, n_campaigns + 1)],
    'categoría': np.random.choice(categories, n_campaigns),
    'país': np.random.choice(countries, n_campaigns),
    'meta_usd': np.random.randint(5000, 50000, n_campaigns),
    'recaudado_usd': lambda df: df['meta_usd'] * np.random.uniform(0.2, 1.3, n_campaigns),
    'fecha_creación': pd.to_datetime('2024-01-01') + pd.to_timedelta(np.random.randint(0, 180, n_campaigns), unit='D')
})
campaigns['recaudado_usd'] = campaigns['meta_usd'] * np.random.uniform(0.2, 1.3, n_campaigns)
campaigns['estado'] = np.where(campaigns['recaudado_usd'] >= campaigns['meta_usd'], 'Meta alcanzada', 'En curso')

# Sidebar
st.sidebar.title("Filtros")
pais = st.sidebar.selectbox("País", ['Todos'] + countries)
categoria = st.sidebar.selectbox("Categoría", ['Todas'] + categories)

# Filtros
df = campaigns.copy()
if pais != 'Todos':
    df = df[df['país'] == pais]
if categoria != 'Todas':
    df = df[df['categoría'] == categoria]

# KPIs
total_recaudado = df['recaudado_usd'].sum()
total_meta = df['meta_usd'].sum()
porcentaje_exito = (df['estado'] == 'Meta alcanzada').mean() * 100
ticket_promedio = df['recaudado_usd'].mean()

st.title("📊 Dashboard de Campañas - GoFundMe (simulado)")

col1, col2, col3, col4 = st.columns(4)
col1.metric("Recaudación total", f"${total_recaudado:,.0f}")
col2.metric("Meta total", f"${total_meta:,.0f}")
col3.metric("Éxito (%)", f"{porcentaje_exito:.1f}%")
col4.metric("Recaudación promedio", f"${ticket_promedio:,.0f}")

# Gráfico: Recaudación por categoría
st.subheader("📁 Recaudación por categoría")
cat_chart = df.groupby('categoría')['recaudado_usd'].sum().reset_index()
st.plotly_chart(px.bar(cat_chart, x='categoría', y='recaudado_usd', title='Recaudación total por categoría'))

# Gráfico: Mapa por país
st.subheader("🌍 Distribución geográfica")
geo = df.groupby('país')['recaudado_usd'].sum().reset_index()
geo['iso'] = geo['país'].map({
    'Argentina': 'ARG',
    'USA': 'USA',
    'Brasil': 'BRA',
    'México': 'MEX',
    'España': 'ESP'
})
st.plotly_chart(px.choropleth(geo, locations='iso', color='recaudado_usd',
                              hover_name='país', title='Recaudación total por país',
                              color_continuous_scale='blues'))

# Tabla de campañas destacadas
st.subheader("⭐ Campañas con mayor recaudación")
top_campañas = df.sort_values('recaudado_usd', ascending=False).head(10)
st.dataframe(top_campañas[['nombre', 'categoría', 'país', 'meta_usd', 'recaudado_usd', 'estado']])

# Alertas
st.subheader("🚨 Campañas cerca de la meta")
casi_meta = df[(df['recaudado_usd'] / df['meta_usd']) >= 0.9 & (df['estado'] != 'Meta alcanzada')]
st.dataframe(casi_meta[['nombre', 'país', 'categoría', 'recaudado_usd', 'meta_usd']])


ModuleNotFoundError: No module named 'streamlit'